In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from sklearn.metrics import f1_score

In [ ]:
import boto3
import botocore

s3 = boto3.resource('s3')
bucket = s3.Bucket('algo-insights-multiclass')
CHECKPOINTS = "multiclass-classifier-checkpoint.pth"
TRAIN_DATA = "multiclass-6labels-training-full.csv"
try:
    bucket.download_file(CHECKPOINTS, CHECKPOINTS)
    bucket.download_file(TRAIN_DATA, TRAIN_DATA)
except botocore.exceptions.ClientError as e:
    print(e)

In [ ]:
train_df = pd.read_csv(TRAIN_DATA)
train_df.head()
# path = "./data/multiclass-6labels-training.csv"
# train_df = pd.read_csv(path)

In [ ]:
codes = {'Movies_Negative':0, 'Movies_Positive':1, 'Food_Negative':2, 'Food_Positive': 3, 'Clothing_Negative': 4, 'Clothing_Positive':5}
train_df['class'] = train_df['Originally'].map(codes)
train_df.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_df['clean_text'], 
                                                  train_df['class'],
                                                  random_state=2018,
                                                  test_size=0.3, 
                                                  stratify=train_df['class'])


In [ ]:
max_seq_len = 250
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                        num_labels = 6,
                                        output_attentions=False,
                                        output_hidden_states=False)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
#model.load_state_dict(torch.load(CHECKPOINTS))
model.to("cuda")

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    X_train.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)
input_ids_train = torch.tensor(encoded_data_train['input_ids'])
attention_masks_train = torch.tensor(encoded_data_train['attention_mask'])
labels_train = torch.tensor(y_train.tolist())


dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)  #to creat the data loaders

In [ ]:
encoded_data_val = tokenizer.batch_encode_plus(
    X_val.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

input_ids_val = torch.tensor(encoded_data_val['input_ids'])
attention_masks_val = torch.tensor(encoded_data_val['attention_mask'])
labels_val = torch.tensor(y_val.tolist())

dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

# Data Loaders

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 32
dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)
dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)


In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
optimizer = AdamW(model.parameters(),
                  lr=2e-5,
                  eps=1e-8)
                  
epochs = 1
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

# Metric Evaluators


In [ ]:
label_dict = codes

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
import random
from tqdm.notebook import tqdm

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):
    model.eval()
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        batch = tuple(b.to("cuda") for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)            
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to("cuda") for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

predictions = np.argmax(predictions, axis = 1)

print('The classification report: ')
target_names = codes.keys()
print(classification_report(true_vals, predictions, target_names=target_names))

In [ ]:
torch.save(model.state_dict(), "distilbert-checkpoints.pth")